In [1]:
#import dependencies
import pandas as pd
import gmaps
import requests
import ipywidgets as widgets

#Import API key
from config import g_key

In [2]:
#Store the saved CSV created in part one into a data frame
city_data_df = pd.read_csv('weather_data/cities.csv')
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Albany,US,2021-10-30 00:25:00,42.6001,-73.9662,49.14,73,23,5.99
1,1,Jesus De Otoro,HN,2021-10-30 00:28:10,14.4833,-87.9833,74.86,99,69,3.29
2,2,Saint-Philippe,RE,2021-10-30 00:28:11,-21.3585,55.7679,69.48,82,42,6.60
3,3,Puerto Ayora,EC,2021-10-30 00:28:11,-0.7393,-90.3518,69.82,91,76,10.07
4,4,Kaitangata,NZ,2021-10-30 00:26:54,-46.2817,169.8464,66.87,57,63,12.97


In [3]:
#Get the data types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
#Configure gmaps to use your Google API key
gmaps.configure(api_key = g_key)

In [8]:
#Get the maximum temperature
max_temp = city_data_df['Max Temp']
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [9]:
#Get the latitude and longitude
locations = city_data_df[['Lat', 'Lng']]

#Assign the figure variable
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#Assign heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp,0) for temp in max_temp], 
                                 dissipating=False, max_intensity=300, point_radius=4)

#Add the heatmap layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
#Percent Humidity Heatmap
locations = city_data_df[['Lat', 'Lng']]
humidity = city_data_df['Humidity']
fig = gmaps.figure(center=(30.0, 31), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
#Percent Cloudiness Heatmap
locations = city_data_df[['Lat', 'Lng']]
clouds = city_data_df['Cloudiness']
fig = gmaps.figure(center=(30.0, 31), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, 
                                 dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Wind Speed heatmap

locations = city_data_df[['Lat', 'Lng']]
wind = city_data_df['Wind Speed']
fig = gmaps.figure(center=(30.0, 31), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, 
                                 dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [13]:
#Ask the customer to add a minimum and maximum temperature value
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 100


In [14]:
#Filter the data set to find the cities that fit the crieteria
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & \
                                       (city_data_df['Max Temp'] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Jesus De Otoro,HN,2021-10-30 00:28:10,14.4833,-87.9833,74.86,99,69,3.29
5,5,Atuona,PF,2021-10-30 00:28:12,-9.8000,-139.0333,77.38,72,24,12.39
6,6,Naze,JP,2021-10-30 00:28:13,28.3667,129.4833,74.32,60,81,10.13
7,7,Hilo,US,2021-10-30 00:28:13,19.7297,-155.0900,82.13,69,1,8.05
8,8,Merauke,ID,2021-10-30 00:25:06,-8.4667,140.3333,83.55,74,100,5.50
10,10,Soyo,AO,2021-10-30 00:28:15,-6.1349,12.3689,77.67,88,82,12.35
11,11,Rikitea,PF,2021-10-30 00:25:08,-23.1203,-134.9692,75.47,79,96,17.67
18,18,Mataura,NZ,2021-10-30 00:28:18,-46.1927,168.8643,71.58,46,53,4.32
19,19,Buala,SB,2021-10-30 00:28:19,-8.1450,159.5921,85.73,71,95,11.32
20,20,Sokoni,TZ,2021-10-30 00:28:19,-6.4333,39.5500,78.84,78,46,8.84


In [15]:

preferred_cities_df.count()

City_ID       215
City          215
Country       214
Date          215
Lat           215
Lng           215
Max Temp      215
Humidity      215
Cloudiness    215
Wind Speed    215
dtype: int64

In [16]:
#Create a data frame called hotel_df to store hotel names along with city, country, max temp, and coordinates
hotel_df = preferred_cities_df[['City', 'Country', 'Max Temp', 'Lat', 'Lng']].copy()
hotel_df['Hotel Name'] = ''
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Jesus De Otoro,HN,74.86,14.4833,-87.9833,
5,Atuona,PF,77.38,-9.8000,-139.0333,
6,Naze,JP,74.32,28.3667,129.4833,
7,Hilo,US,82.13,19.7297,-155.0900,
8,Merauke,ID,83.55,-8.4667,140.3333,
10,Soyo,AO,77.67,-6.1349,12.3689,
11,Rikitea,PF,75.47,-23.1203,-134.9692,
18,Mataura,NZ,71.58,-46.1927,168.8643,
19,Buala,SB,85.73,-8.1450,159.5921,
20,Sokoni,TZ,78.84,-6.4333,39.5500,


In [17]:
#Set parameters to search for a hotel
params = {
    'radius': 5000,
    'type': 'lodging',
    'key': g_key
}

#Iterate through the data frame
for index, row in hotel_df.iterrows():
    #Get the latitude and longitude
    lat = row['Lat']
    lng = row['Lng']
    
    #Add the latitude and longitude to location key for the params dictionary
    params['location'] = f"{lat},{lng}"
    
    #Use the search term: "lodging" and our latitude and longitude
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    #Make request and get the JSON data from the search
    hotels = requests.get(base_url, params=params).json()
    #Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping.')

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [18]:

hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Jesus De Otoro,HN,74.86,14.4833,-87.9833,Hotel Tosta
5,Atuona,PF,77.38,-9.8000,-139.0333,Villa Enata
6,Naze,JP,74.32,28.3667,129.4833,オーベルジュ・アガルイ
7,Hilo,US,82.13,19.7297,-155.0900,Hilo Hawaiian Hotel
8,Merauke,ID,83.55,-8.4667,140.3333,
10,Soyo,AO,77.67,-6.1349,12.3689,Hotel Mpampa
11,Rikitea,PF,75.47,-23.1203,-134.9692,People ThankYou
18,Mataura,NZ,71.58,-46.1927,168.8643,Ellie's Villa
19,Buala,SB,85.73,-8.1450,159.5921,Maringe Lagoon Lodge
20,Sokoni,TZ,78.84,-6.4333,39.5500,Ujamaa Beach Resort


In [19]:
#Add a heatmap of temperature for vacation spots
locations = hotel_df[['Lat', 'Lng']]
max_temp = hotel_df['Max Temp']
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, 
                                 max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

#Store the data frame row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]



In [23]:

#Add a heatmap of temperature for vacation spots and a pop-up marker for each city
locations = hotel_df[['Lat', 'Lng']]
max_temp = hotel_df['Max Temp']
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, 
                                 max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))